In [1]:
# load and evaluate a saved model
from numpy import loadtxt
from tensorflow.keras.models import load_model
import os
import pandas as pd
import numpy as np # linear algebra


from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
# # load model
# model = load_model('lstm_model.h5')
# # summarize model.
# model.summary()
# # load dataset
# dataset = loadtxt("nlp_research_dataset_1000.csv")
# # split into input (X) and output (Y) variables
# X = dataset[:,0:]
# Y = dataset[:,8]
# # evaluate the model
# score = model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [2]:
data = pd.read_csv('Validation/final_clean.csv')

In [3]:
loaded_model = load_model('lstm_model.h5')

In [4]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [5]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 34, 128)           256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 34, 128)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
data.head()

,text,Unnamed: 1,sentiment
0,lunes nanaman bukas academicbreaknow,NaN,0
1,academicbreaknow q tapusin wednesday,NaN,0
2,super delay tbw list academicbreaknow,NaN,0
3,academicbreaknow tsngina pagod magpaypay,NaN,0
4,pagoda accla hours tulog gising lecture practi...,NaN,0


In [8]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(670, 34) (670, 2)
(330, 34) (330, 2)


In [11]:
validation_size = 1500
batch_size = 32
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

C:\Users\Administrator\anaconda3\lib\site-packages\keras\utils\generic_utils.py:981: RuntimeWarning: divide by zero encountered in log10
  numdigits = int(np.log10(self.target)) + 1


OverflowError: cannot convert float infinity to integer